#### Lab 01 - Content Based Recommender (Film Genre)

Maintain User Engagement by Generating Recommendation Using Film Genre

Tackle Cold Start Problem by Asking User Preferences

In [ ]:
import warnings, pandas

warnings.filterwarnings("ignore")

In [ ]:
# Use Movies Table on Origin

origin = pandas.read_table("movies.csv", sep=",")

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
origin.tail()

,MovieID,Title,Genre
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [ ]:
null = origin.isnull().sum()

null.sum()

0

In [ ]:
total_film = origin.shape[0]

total_film

# 9.742

9742

Total Film is 9.742

#### Maintain Genre Column Quality

In [ ]:
# Change Genre Column Separator

splita = lambda val : " ".join(val.split("|"))

origin["Genre"] = origin["Genre"].apply(splita)

In [ ]:
# Delete Film Without Genre

nolist = "(no genres listed)"

origin = origin[origin["Genre"] != nolist]

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


For Our Goal Using `TfidfVectorizer`, Replacing `Sci-Fi` to `SciFi`, `Film-Noir` to `Filnoir` is Mandatory

Others ? Resulting `Sci-Fi` to (`Sci` + `Fi`), `Film-Noir` to (`Film` + `Noir`)

In [ ]:
# Replace Sci-Fi + Film-Noir

replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  origin["Genre"] = origin["Genre"].str.replace(i, t)

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Find List of Genres

ListOfGenre = []

for insect in origin["Genre"].values:

  insect = insect.split(" ")

  for val in insect:

    if val not in ListOfGenre: ListOfGenre.append(val)

len(ListOfGenre)

19

Total Genre is 19

In [ ]:
# Check Our First Five Genres

ListOfGenre[:5]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

In [ ]:
# Converting Genres to TF-IDF (Term Frequency-Inverse Document Frequency) Matrix

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")

metrics = tfidf.fit_transform(origin["Genre"])

metrics.shape

(9708, 19)

Our TF IDF Metrics Consist of 9708 Film and 19 Genres

In [ ]:
# Understand our TF IDF Metrics (1)

tfidf.get_feature_names_out().tolist()[:5]

['action', 'adventure', 'animation', 'children', 'comedy']

In [ ]:
# Understand our TF IDF Metrics (2)

len(tfidf.get_feature_names_out().tolist())

19

In [ ]:
# Understand our TF IDF Metrics (3)

assert len(tfidf.get_feature_names_out().tolist()) == len(ListOfGenre), "Fail !"

In [ ]:
# Check Genre : Comedy (5), Romance (15), Drama (8)

metrics.todense()[3:4]

matrix([[0.        , 0.        , 0.        , 0.        , 0.50488626,
         0.        , 0.        , 0.46621628, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.72645182,
         0.        , 0.        , 0.        , 0.        ]])

In [ ]:
origin.iloc[3:4]

,MovieID,Title,Genre
3,4,Waiting to Exhale (1995),Comedy Drama Romance


In [ ]:
tempor = tfidf.get_feature_names_out().tolist()

tempor[4], tempor[7], tempor[14]

('comedy', 'drama', 'romance')

Perform Cosine Similarity to Measure Genres Similarity Between One Film to Another Film

In [ ]:
# Use Cosine Similarity to Find Similarity of Our TF IDF Metrics

from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(metrics)

similarities.shape

(9708, 9708)

In [ ]:
# Check Similarities (1)

len(similarities[0])

9708

In [ ]:
# Jumanji Similarities Result

similarities[1][:5]

array([0.8136036, 1.       , 0.       , 0.       , 0.       ])

In [ ]:
origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


Our Movies, Jumanji, Have 81 % Similarity on Genres to Toy Story

In [ ]:
# Helper Function

intler = lambda i : origin.iloc[i, 1]

titint = lambda t : origin[origin["Title"] == t].index.values[0]

In [ ]:
# Helper Test

title = "Jumanji (1995)"

titint(title)

1

#### Creating Main Function to Our Model !

In [ ]:
similar_result = enumerate(similarities[int(titint(title))])

orilist = list(similar_result)

len(orilist)

9708

In [ ]:
orilist[:5]

[(0, 0.8136035975025231), (1, 1.0), (2, 0.0), (3, 0.0), (4, 0.0)]

In [ ]:
orilist = sorted(orilist, key=lambda i : i[1], reverse=True)

orilist[:5]

[(1, 1.0), (53, 1.0), (109, 1.0), (767, 1.0), (1514, 1.0)]

In [ ]:
origin[origin.index.isin([i[0] for i in orilist[:5]])]

,MovieID,Title,Genre
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy


#### Main Model

In [ ]:
# Create Main Model Output Function

limit = 5

title = "Jumanji (1995)"

def output(title, limit):

  morate = enumerate(similarities[int(titint(title))])
  molist = list(morate)
  morter = sorted(molist, key=lambda i:i[1], reverse=True)

  molist = list(morter); lite = limit + 1
  molist = filter(lambda i : i[0] != titint(title), molist[:lite])
  titles = [origin.iloc[i[0], 1] for i in molist]

  return origin[origin["Title"].isin(titles)].iloc[:limit]

output(title, limit)

,MovieID,Title,Genre
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy
1556,2093,Return to Oz (1985),Adventure Children Fantasy


In [ ]:
title = origin.iloc[3574, 1]

output(title, 5)

,MovieID,Title,Genre
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy


#### First Time User

`Cold Start Problem`

On Our Solution, Cold Start Problem Can be Tackle with Asking User Genre Preference

Let's Try Tackle Cold Start Problem on Michael, Our Customers

In [ ]:
# Michael Recently Subscribe to Our Product

# Michael Choose Children and Fantasy Genres

option = ["Children", "Fantasy"]

option

['Children', 'Fantasy']

In [ ]:
# Find Film to Recommend to Michael

def starter(option, limit):

  result = []

  ontari = origin["Genre"].unique()

  for item in ontari:
    # Split Genre
    spliter = item.split(" ")

    # Find Overlap Between Genre Input and Available Genre
    overlap = len(set(option) & set(spliter)) / len(spliter)
    if overlap > 0.5:
      result.append((item, overlap))

  # Sort Table Result
  result = sorted(result, key=lambda i:i[1], reverse=True)[:limit]
  result = [i[0] for i in result]
  result = origin[origin["Genre"].isin(result)].iloc[:limit]

  return result

starter(option, 5)

,MovieID,Title,Genre
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
209,243,Gordy (1995),Children Comedy Fantasy
301,343,"Baby-Sitters Club, The (1995)",Children


Michael Choose The Indian in the Cupboard

Let's Recommend Other Film to Michael

In [ ]:
# Recommend Other Film to Michael Based on The Indian in the Cupboard

title = "Indian in the Cupboard, The (1995)"

output(title, 5)

,MovieID,Title,Genre
1,2,Jumanji (1995),Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy
1556,2093,Return to Oz (1985),Adventure Children Fantasy
